In [24]:
import torch
import torch.nn as nn
import time

# Tạo dữ liệu huấn luyện và phân loại
inputs = torch.randn(100, 784) # tạo dữ liệu huấn luyện với kích thước 100x784
labels = torch.randint(10, (100,)) # tạo dữ liệu phân loại với kích thước 100

# Thiết kế mô hình
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(784, 128) # số lượng đầu vào 784, số lượng đơn vị trong lớp ẩn 128
        self.fc2 = nn.Linear(128, 10) # số lượng đơn vị trong lớp ẩn 128, số lượng đầu ra 10

    def forward(self, x):
        x = torch.relu(self.fc1(x)) # áp dụng hàm kích hoạt ReLU cho lớp ẩn
        x = self.fc2(x)
        return x

# Tạo instance của mô hình
model = Model()

# Huấn luyện mô hình
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

for epoch in range(10):
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

print("Mô hình đã được huấn luyện!")

# Tạo dữ liệu thử nghiệm
test_inputs = torch.randn(100, 784)
test_labels = torch.randint(10, (100,))

# Lập trình đánh giá
model.eval()
with torch.no_grad():
    test_outputs = model(test_inputs)

criterion = nn.CrossEntropyLoss()
test_loss = criterion(test_outputs, test_labels).item()

start_time = time.time()
# Huấn luyện mô hình...
test_outputs = model(test_inputs)
end_time = time.time()

# Đo lường hiệu suất
accuracy = torch.sum(torch.argmax(test_outputs, dim=1) == test_labels).item() / len(test_labels)
print("Độ chính xác:", accuracy * 100)
print("Thời gian huấn luyện:", time.time() - start_time)
print("Thời gian đánh giá:", time.time() - end_time)
print("Thời gian tổng:", time.time() - start_time)

Mô hình đã được huấn luyện!
Độ chính xác: 13.0
Thời gian huấn luyện: 0.0003037452697753906
Thời gian đánh giá: 0.00016307830810546875
Thời gian tổng: 0.00035381317138671875
